# FT-ICR analysis - PART 1

Created on 01 March 2020 for the Pentatrap experiment

@author: Menno Door<br>
@contact: door+fticr@mpi-k.de<br>
@license: MIT license

### Refenences :

used toolkit of Jonas Karthein for PI-ICR as template :) thank you!!<br>

For references reguarding the theory behind ft-icr detection and analysis methods, please check the references.md file.

### Introduction

This part of the analysis does the conversion of the raw data to penning trap eigenfrequency data. You will be able to group, filter and average data and then fit spectra and/or evaluate pnp phase data to determine these frequencies.

### Important remark:

This is more a visualization thing, especially for bigger data sets i would recommend to do this using a normal python script including some multiprocessing.

Please be patient with this one! Gathering information on paths in fhds structures may take a while, loading data too. So please look at the circle on the top right corner (jupyter notebook), if its filled, there is still something going on and you should just wait.

### Requirements:

The following code was written for Python 3.7/3.8. The required libraries are listed below with a rough description for their task in the code.

    fhds (inhouse data storage, https://git.mpi-hd.mpg.de/Pentatrap/fhds)
    h5py (data storage, easier to distribute)
    pandas (data organisation, calculation and visualization)
    numpy (calculation)
    matplotlib (plotting)
    scipy (chi square fitting)
    jupyter (Python notebook environment)
    ipywidgets (https://ipywidgets.readthedocs.io/en/latest/user_install.html)
    plotly (plotting, https://github.com/plotly/plotly.py#installation)
    qgrid (data visualization, https://github.com/quantopian/qgrid#installation)

### ToDo 

- check for phase correlation here with single phases, short and long phases and unwrap.

In [ ]:
# standard libs
from datetime import datetime
from pathlib import Path
import os, json
from pprint import pprint

# math and data
import numpy as np
import scipy
import pandas as pd
import h5py

# visualization
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
#display(HTML("<style>div.output_scroll { height: 150em; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
%matplotlib inline
import matplotlib.pyplot as plt
import qgrid
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
py.init_notebook_mode()
plt.rcParams['figure.figsize'] = (10, 4)

%load_ext autoreload
%autoreload 2

# this package
from fhds import fhds, interface
from fticr_toolkit import *
#from trap_control import fitaxialClass

### Step 1: Load raw data and inspect

The input_data filename should be either a folder following fhds rules (append ".fhds" to the folder name please) or a hdf5 file.

In [ ]:
###   P A R A M E T E R   L I S T   ( this is especially used for batch processing using the papermill package )
measurement_folder = "Z:\Measurements\\174Yb41+_unity_feedback_3khz_2"
input_file = 'pnp_dip_unwrap.fhds'
output_folder = "./part1_data/"
use_settings_file = False
settings = {
    "nu_unwrap": "nu_p",
    "unwrap_range": 2,
    "grouping": [10],
    "reuse_averages": True,
    "reuse_fitdata": True,
    "fixed_phase_readout": False
}

In [ ]:

data, meas_config, output_folder = data_conversion.load_data(measurement_folder=measurement_folder, 
                          input_data=input_file, 
                          output_folder="part1_data",
                          measurement_script = "unwrap_n_measure_repeat"
                         )

if use_settings_file:
    settings.update( data_conversion.load_settings(measurement_folder) )
print("settings", settings)

#pprint(meas_config)
data_root = data["meas_data"]

In [ ]:
####
# Chech the data structure, cycles and such
#
####

mcycles = list(data_root.keys())
print("root group names / MAIN CYCLES : ", mcycles )

measurement_types = list( data_root[mcycles[0]].keys() )
print("check out first group / MEASUREMENT TYPES : ", measurement_types )
if not 'pre_unwrap' in measurement_types:
    raise TypeError("There is no pre unwrap data here!")

raw_data_keys = data_conversion.get_data_paths( data_root, name_includes= ("nu_z", "phase") )

### now unwrap data...

In [ ]:

pre_unwrap_phase = pd.DataFrame( columns=[ "mcycle", "cycle", "position", "trap", "acc_time", "time", "phase" ] )
only_attr = {"type": "phase"}

for mc in data_root.keys():
    unwrap_data = data_root[mc]["pre_unwrap"]
    for subcycle, subcycle_grp in unwrap_data.items():
        for pos, position_grp in subcycle_grp.items():
            for trap, trap_grp in position_grp.items():
                print("\nsubcycle", subcycle, "pos", pos, "trap", trap)#, end="\r")

                for name, dset in trap_grp.items():
                    # check if this is the correct type of data to average
                    skip = False
                    for attr, val in only_attr.items():
                        if dset.attrs[attr] != val:
                            skip = True
                    if skip:
                        continue
                    
                    # NOTE: this here is sadly very important: the dset[:] hhas to come before
                    #       getting the attributes, due to some automated update in fhds
                    dset_data = dset[:]
                    dset_attrs = {}
                    for key, val in dset.attrs.items():
                        dset_attrs[key] = val
                        
                    freqs = np.linspace(dset_attrs["start"], dset_attrs["stop"], dset_attrs["binnum"])

                    nu_z = meas_config[pos]["configuration"]["traps"][int(trap[4:])]["nu_z"]
                    
                    idx = (np.abs(freqs - nu_z)).argmin()
                    phase = dset_data[1][idx]
                    
                    # add they data
                    dset = pd.Series(data=[ int(mc[5:]), int(subcycle[5:]), pos, int(trap[4:]), dset_attrs["acc_time"], dset_attrs["time"], phase ] , index=pre_unwrap_phase.columns)
                    pre_unwrap_phase = pre_unwrap_phase.append(dset, ignore_index=True)

                    print('.', end='')

In [ ]:
pre_unwrap_phase["time"] = pd.to_datetime(pre_unwrap_phase['time'], format='%Y%m%d_%H%M%S.%f')
display(pre_unwrap_phase)
pre_unwrap_phase.to_csv(output_folder + "/pre_unwrap_phase.csv", index=False)

## Check data

In [ ]:
print(" >>> UNWRAP DATA >>> ")
pre_unwrap_phase["acc_time"] = pre_unwrap_phase["acc_time"].astype(str)
fig = px.scatter(pre_unwrap_phase, x="time", y="phase", facet_col="trap", facet_row="position", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
#fig.update_yaxes(matches=None)
fig.show()
pre_unwrap_phase["acc_time"] = pre_unwrap_phase["acc_time"].astype(float)

### Step 4: N determination

The unwrap data (in this case just for the nu_p phase measurement) is used to calculate the total N of osciallations during the phase accumulation time in the later measurement

The method n_determination.fit_N will unwrap the phases for each acc_time, average, substract the reference phases from all the other measured phases and then determine the N by calculating Ns for a 1 Hz range around the guessed frequency and searches for the minimum. A plot of the Ns and the found minimum will be plotted to check the results.

The result will be a dataFrame including N, end_phase and frequency for all main cycles, traps and positiions.

In [ ]:
# each ion in each trap its own N determination, for every main cycle! (no grouping here, wouldn't make any sense)

columns = ["mcycle", "trap", "position", "N", "end_phase", "nu_p", "ion", "time", "max_acc_time"]
nu_p_N = pd.DataFrame(columns = columns)

for name, subset in pre_unwrap_phase.groupby(["mcycle", "trap", "position"]):
    mc, trap, pos = name
    
    # NOTE: if the structure of the config changed, you have to adjust here!
    nu_guess = meas_config[pos]["configuration"]["traps"][trap][settings["nu_unwrap"]]
    ion_str = meas_config[pos]["configuration"]["traps"][trap]["ion"]
    #evolution_time = abs(meas_config["accumulation_time"][0]["time"] - meas_config["accumulation_time"][1]["time"])

    print(" >>> mc", mc, "trap", trap, "pos", pos, " <<< ")
    N, end_phase, nu_p, mean_time, max_acc_time = phase_analysis.determine_N(subset, nu_guess, resolution=None, nu_range=settings["unwrap_range"], show=True)
    new_row = pd.Series([mc, trap, pos, N, end_phase, nu_p, ion_str, mean_time, max_acc_time], index=nu_p_N.columns )
    nu_p_N = nu_p_N.append(new_row, ignore_index=True)

# show results and save to csv in results folder
display(nu_p_N)
